In [1]:
import pandas as pd
import glob 
import numpy as np

In [2]:
Path = r'E:\_Projects\US_Shipment_Bklg_Forcast\\'

## Load Cost File

In [3]:
cost = pd.read_excel(Path+'Input\\Cost\\Cost.xlsx')

cost = cost[cost['Item'].str.contains('MC', na=False)]

In [4]:
cost = cost[['IO_Concat','IR_Concat','Number','Item','Price','Order Number']]

In [5]:
#### Change to number
val = ['Number','Order Number']
for x in val:
    cost[x] = cost[x].astype('Int64')

In [6]:
val = ['IO_Concat','IR_Concat','Number','Item','Order Number']

for x in val:
    cost[x] = cost[x].astype(str)


In [7]:
val = ['IO_Concat','IR_Concat']
cost.drop_duplicates(subset=val,keep='first',inplace=True)

val = {'Number':'IR No.', 'Order Number':'IO No.','Item':'PID'}
cost.rename(columns=val, inplace=True)


cost['PID Class'] = cost['PID'].str[:4]
cost['PID Class'] = cost['PID Class'].astype(str)

In [8]:
cost_IO_PID = cost.copy()
cost_IR_PID = cost.copy()
cost_IO = cost.copy()
cost_IR = cost.copy()
cost_MCID = cost.copy()
cost_class = cost.copy()

In [9]:
cost_IO_PID = cost_IO_PID[['IO_Concat','Price']]
cost_IO_PID = cost_IO_PID.drop_duplicates(subset=['IO_Concat'],keep='first')

cost_IR_PID = cost_IR_PID[['IR_Concat','Price']]
cost_IR_PID = cost_IR_PID.drop_duplicates(subset=['IR_Concat'], keep='first')

cost_IO = cost_IO[['IO No.','Price']]
cost_IO = cost_IO.drop_duplicates(subset=['IO No.'],keep='first')

cost_IR = cost_IR[['IR No.','Price']]
cost_IR = cost_IR.drop_duplicates(subset=['IR No.'], keep='first')

cost_MCID = cost_MCID.groupby('PID').agg({'Price': 'mean'}).reset_index()

cost_class = cost_class[['PID Class','Price']]

cost_class = cost_class.groupby('PID Class').agg({'Price':'mean'}).reset_index()

In [10]:
###### Rename Price Columns
cost_IO_PID.rename(columns={'Price':'Price_1'}, inplace=True)
cost_IR_PID.rename(columns={'Price':'Price_2'}, inplace=True)
cost_IO.rename(columns={'Price':'Price_3'}, inplace=True)
cost_IR.rename(columns={'Price':'Price_4'}, inplace=True)
cost_MCID.rename(columns={'Price':'Price_5'}, inplace=True)
cost_class.rename(columns={'Price':'Price_6'}, inplace=True)

## ************************************* Global Doc *************************************

In [11]:
print('**************************************************************************************\n')
print('                 Global Doc is Started ( Backlog Process................!)\n')
print('**************************************************************************************\n')

**************************************************************************************

                 Global Doc is Started ( Backlog Process................!)

**************************************************************************************



In [12]:
####### Read Global DOC
doc_path = Path + 'Input\\'
file_list = glob.glob(doc_path + 'Global DOC Atleos ????????.xlsb')
if file_list:
    Doc = pd.read_excel(file_list[0],sheet_name='Orders',engine='pyxlsb')  
else:
    print("No Glob Doc file found.")

In [13]:
Doc = Doc[(Doc['Org Code'] == 'US') & (Doc['PID_type'] == 'M') & (Doc['ProdRange'] == 'ATM')]


In [14]:
val = ['IR No.','IO No.']

for x in val:
    Doc[x] = Doc[x].astype('Int64')

val = ['IR No.','IO No.']

for x in val:
    Doc[x] = Doc[x].astype(str)

### Including Cost Details

In [15]:
Doc['IO_Concat'] = Doc['IO No.']+Doc['PID']
Doc['IR_Concat'] = Doc['IR No.']+Doc['PID']

In [16]:
Doc = pd.merge(Doc, cost_IO_PID, on='IO_Concat', how='left')

Doc = pd.merge(Doc, cost_IR_PID, on='IR_Concat', how='left')

Doc = pd.merge(Doc, cost_IO, on='IO No.', how='left')

Doc = pd.merge(Doc, cost_IR, on='IR No.', how='left')

Doc = pd.merge(Doc, cost_MCID, on='PID', how='left')

Doc = pd.merge(Doc, cost_class, on='PID Class', how='left')


In [17]:
Doc['Price'] = (
    Doc['Price_1']
    .fillna(Doc['Price_2'])
    .fillna(Doc['Price_3'])
    .fillna(Doc['Price_4'])
    .fillna(Doc['Price_5'])  
    .fillna(Doc['Price_6'])  
    .fillna(0)
)                       

In [18]:
val = ['Price_1','Price_2','Price_3','Price_4','Price_5','Price_6']
try:
    Doc.drop(columns=val,inplace=True)
except:
    pass

In [19]:
Doc['Qty'] = Doc['Qty'].astype('Int64') 
Doc['Cost'] = Doc['Qty'] * Doc['Price']
Doc.rename(columns={'Cost':'ITEM COST'},inplace=True)

In [20]:
#Doc.to_csv(r'doc.csv',index=False)

In [21]:
Pivot = Doc.copy()

#### Grouping

In [22]:

pivot_qty = Pivot.pivot_table(
    index=['Whse', 'ProdRange3', 'PID Class', 'Key Account', 'PID', 'DEMAND VAL'],
    columns='Ship Month',
    values='Qty',
    aggfunc='sum',
    fill_value=0
)


cost_sum = Doc.groupby(['Whse', 'ProdRange3', 'PID Class', 'Key Account', 'PID', 'DEMAND VAL'])['ITEM COST'].sum()


Final_Pivot = pivot_qty.merge(cost_sum, on=['Whse', 'ProdRange3', 'PID Class', 'Key Account', 'PID', 'DEMAND VAL']).reset_index()

In [23]:
Final_Pivot.insert(6,'ITEM COST',Final_Pivot.pop('ITEM COST'))

val = {'Whse':'Plant','ProdRange3':'Offer PF','PID Class':'Class','Key Account':'Key Acc','PID':'MCID','DEMAND VAL':'Order Number','ITEM COST':'Cost'}

Final_Pivot.rename(columns=val, inplace=True)

Final_Pivot['Data series'] = 'BLG'
Final_Pivot['$ ASP'] = None

Final_Pivot.insert(0,'Data series',Final_Pivot.pop('Data series'))
Final_Pivot.insert(9,'$ ASP',Final_Pivot.pop('$ ASP'))


### Qty Pivot 

In [24]:
Pivot = Doc.copy()

Pivot['Data Series'] = 'BKG'

pivot_qty = Pivot.pivot_table(
    index=['Data Series','Whse'],
    columns='Ship Month',
    values='Qty',
    aggfunc='sum',
    fill_value=0
).reset_index()


In [25]:
pivot_qty

Ship Month,Data Series,Whse,202512,202601,202602,202603,202604,202605,202606,202607,202608
0,BKG,CHE,1034,2295,1089,782,421,184,254,161,95
1,BKG,ENN,140,305,175,84,0,2,0,0,0


#### $ Value Cost Pivot

In [26]:
Pivot = Doc.copy()

Pivot['Data Series'] = 'BKG'
Pivot['ITEM COST'] = Pivot['ITEM COST'].round(0)

pivot_cost = Pivot.pivot_table(
    index=['Data Series','Whse'],
    columns='Ship Month',
    values='ITEM COST',
    aggfunc='sum',
    fill_value=0
).reset_index()


for col in pivot_cost.columns:
    if col not in ['Data Series', 'Whse']:
        pivot_cost[col] = pivot_cost[col].apply(lambda x: f"${x:,}")



In [27]:
pivot_cost

Ship Month,Data Series,Whse,202512,202601,202602,202603,202604,202605,202606,202607,202608
0,BKG,CHE,"$7,450,579.0","$16,661,608.0","$10,119,179.0","$5,367,165.0","$2,760,205.0","$1,398,548.0","$1,859,768.0","$1,117,240.0","$746,848.0"
1,BKG,ENN,"$2,070,204.0","$5,135,764.0","$3,267,202.0","$1,614,073.0",$0.0,"$34,696.0",$0.0,$0.0,$0.0


In [28]:
Final_Pivot.to_excel(Path+'Output\\Summary\\Backlog by Summary.xlsx',index=False)
pivot_qty.to_excel(Path+'Output\\Mail\\Backlog by Qty.xlsx',index=False)
pivot_cost.to_excel(Path+'Output\\Mail\\Backlog by Cost.xlsx',index=False)

In [29]:
print('**************************************************************************************\n')
print('                 Global Doc is Completed ( Backlog Process................!)\n')
print('**************************************************************************************\n')

**************************************************************************************

                 Global Doc is Completed ( Backlog Process................!)

**************************************************************************************



## ***************************************************************************************************************

## ******************************** Shipment File ********************************

In [30]:
print('**************************************************************************************\n')
print('                 Shipment Process................!)\n')
print('**************************************************************************************\n')

**************************************************************************************

                 Shipment Process................!)

**************************************************************************************



In [31]:
Data = pd.read_csv(Path+'Input\\Results.csv')
offer = pd.read_csv(Path+'Input\\class_offerpf_map.csv')

In [32]:
offer = offer.drop_duplicates(subset=['Class'], keep='first')
Data = Data.drop_duplicates(subset=['Inventory Item Serial Number'], keep='first')


#### Rename Columns 

In [33]:
val = {'Account Sales Order Number':'Order Number','Plant Code':'Plant Code','Inventory Item Serial Number':'Serial Number','Inventory Item Object Identifier':'MCID','Account Ship to Country Code':'Country Code','Account Name':'Master Customer','Account Number':'Master Customer Number','Actual Ship Date':'Actual Ship Date','Inventory Organization Code':'Plant','Ship from Inventory Organization Name':'Area'}

Data.rename(columns=val,inplace=True)

### Update Key Account

In [34]:
############################# PUll KA File from Orale SQL #########################

import oracledb

SQL_Data = 0

if SQL_Data == 0:
    
    print('Pulling KA from Oracle SQL...........!')

    # Enable thick mode
    oracledb.init_oracle_client(lib_dir=r"E:\softwares\instantclient-basic-windows.x64-23.5.0.24.07\instantclient_23_5")

    # Connection details
    dsn_tns = oracledb.makedsn('153.84.75.216', 1521, service_name='ebs_ATMPROD')


    # Use a context manager to handle the connection
    with oracledb.connect(user='erp_user', password='erp_pass', dsn=dsn_tns) as conn:
        cust = pd.DataFrame()

        query = f"""
        select * from
        NCR_DEM_KEY_ACCOUNT_ASSG
        """

        # Fetch data into a pandas DataFrame and append to the main DataFrame
        KA = pd.read_sql(query, con=conn)

elif SQL_Data == 1:
    KA = pd.read_csv(Path+'Input\\SQL.csv')
else:    
    print('SQL Error')


KA['MASTER_CUST_NUM'] = KA['MASTER_CUST_NUM'].astype(str)

value = {'MASTER_CUST_NUM':'Master Customer Number','KEY_ACCOUNT':'Key Acc'}
KA.rename(columns=value, inplace=True)

KA = KA[['Master Customer Number','Key Acc']]

Data['Master Customer Number'] = pd.to_numeric(Data['Master Customer Number'], errors='coerce').astype('Int64')
Data['Master Customer Number'] = Data['Master Customer Number'].astype(str)
Data = pd.merge(Data,KA, on='Master Customer Number', how='left')

Data.loc[Data['Key Acc'].isnull(),'Key Acc'] = 'OTHER SPECIALITY RETAIL'


Pulling KA from Oracle SQL...........!


C:\Users\nn385006\AppData\Local\Temp\2\ipykernel_18372\923548578.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  KA = pd.read_sql(query, con=conn)


#### Plant Update

In [35]:
conditions = [
    Data['Plant Code'] == 94,
    Data['Plant Code'] == 71,
    Data['Plant Code'] == 57,
    Data['Plant Code'] == 76
]

choices = ['CHE', 'CHS', 'JAB', 'EC']

Data['Plant'] = np.select(conditions, choices, default='New_plant')

Data = Data[Data['Plant']!='New_plant']

In [36]:
Data = Data[Data['Country Code']=='US']

Data['Class'] = Data['MCID'].str[:4]
#Data['Class'] = Data['Class'].astype('Int64')
Data['Class'] = Data['Class'].astype(str)

Data['ASD Mth'] = pd.to_datetime(Data['Actual Ship Date'], format='%Y-%m-%d').dt.strftime('%Y%m')
#Data['ASD Mth'] = pd.to_datetime(Data['Actual Ship Date'], format='%d-%m-%Y').dt.strftime('%Y%m')


Data['Qty'] = 1

In [37]:
Data = pd.merge(Data,offer, on='Class',how='left')

Data = Data[
    (Data['Prod Grp'] == 'ATM') &
    (Data['Offer PF'].str.contains('CD|MF', na=False))
]


### Including Cost 

In [38]:
val = {'Order Number':'IO No.','MCID':'PID','Class':'PID Class'}
Data.rename(columns=val,inplace=True)

Data['IO No.'] = Data['IO No.'].astype('Int64')
Data['IO No.'] = Data['IO No.'].astype(str)


Data['IO_Concat'] = Data['IO No.']+Data['PID']


In [39]:
Data = pd.merge(Data, cost_IO_PID, on='IO_Concat', how='left')

Data = pd.merge(Data, cost_IO, on='IO No.', how='left')

Data = pd.merge(Data, cost_MCID, on='PID', how='left')

Data = pd.merge(Data, cost_class, on='PID Class', how='left')


In [40]:
Data['Price'] = (
    Data['Price_1']
    .fillna(Data['Price_3'])
    .fillna(Data['Price_5'])  
    .fillna(Data['Price_6'])  
    .fillna(0)
)                       

In [41]:
val = ['Price_1','Price_3','Price_5','Price_6']
try:
    Data.drop(columns=val,inplace=True)
except:
    pass

In [42]:
Data['Qty'] = Data['Qty'].astype('Int64') 
Data['Price'] = Data['Qty'] * Data['Price']


val = {'IO No.':'Order Number','PID':'MCID','PID Class':'Class','Price':'Cost'}
Data.rename(columns=val,inplace=True)

### Grouping 

In [43]:
Pivot = Data.copy()

pivot_qty = Pivot.pivot_table(
    index=['Plant','Offer PF','Class','Key Acc','MCID','Order Number'],
    columns='ASD Mth',
    values='Qty',
    aggfunc='sum',
    fill_value=0
).reset_index()


cost_sum = Data.groupby(['Plant','Offer PF','Class','Key Acc','MCID','Order Number'])['Cost'].sum().reset_index()


Final_Pivot = pivot_qty.merge(cost_sum, on=['Plant','Offer PF','Class','Key Acc','MCID','Order Number'])

In [44]:
Final_Pivot.insert(6,'Cost',Final_Pivot.pop('Cost'))
Final_Pivot['Data series'] = 'SHP'

Final_Pivot.insert(0,'Data series',Final_Pivot.pop('Data series'))


### Qty Pivot

In [45]:
Pivot = Data.copy()

Pivot['Data Series'] = 'SHP'

pivot_qty = Pivot.pivot_table(
    index=['Data Series','Plant'],
    columns='ASD Mth',
    values='Qty',
    aggfunc='sum',
    fill_value=0
).reset_index()


In [46]:
pivot_qty

ASD Mth,Data Series,Plant,202501,202502,202503,202504,202505,202506,202507,202508,202509,202510,202511,202512
0,SHP,CHE,212,388,1043,372,100,385,130,474,1261,1023,1760,390
1,SHP,CHS,243,495,810,739,1041,1117,540,272,590,76,379,0
2,SHP,EC,76,174,190,207,164,95,200,140,68,45,0,0
3,SHP,JAB,46,70,4,8,6,18,0,4,1,0,1,0


#### $ Value Cost Pivot

In [47]:
Pivot = Data.copy()

Pivot['Data Series'] = 'SHP'
Pivot['Cost'] = Pivot['Cost'].round(0)

pivot_cost = Pivot.pivot_table(
    index=['Data Series','Plant'],
    columns='ASD Mth',
    values='Cost',
    aggfunc='sum',
    fill_value=0
).reset_index()


for col in pivot_cost.columns:
    if col not in ['Data Series', 'Plant']:
        pivot_cost[col] = pivot_cost[col].apply(lambda x: f"${x:,}")



In [48]:
pivot_cost

ASD Mth,Data Series,Plant,202501,202502,202503,202504,202505,202506,202507,202508,202509,202510,202511,202512
0,SHP,CHE,"$926,434.0","$1,796,504.0","$4,449,925.0","$1,524,506.0","$399,609.0","$1,587,513.0","$552,451.0","$2,071,123.0","$5,492,630.0","$4,274,005.0","$7,676,679.0","$1,659,060.0"
1,SHP,CHS,"$2,876,748.0","$6,111,636.0","$9,944,512.0","$9,298,135.0","$13,357,427.0","$13,810,817.0","$6,439,369.0","$3,233,993.0","$7,428,610.0","$940,203.0","$4,401,775.0",$0.0
2,SHP,EC,"$1,433,908.0","$3,106,452.0","$3,090,834.0","$3,659,147.0","$2,376,962.0","$1,456,848.0","$2,885,837.0","$2,084,982.0","$1,284,546.0","$745,824.0",$0.0,$0.0
3,SHP,JAB,"$753,521.0","$1,116,650.0","$56,618.0","$126,176.0","$85,009.0","$285,798.0",$0.0,"$46,670.0","$15,529.0",$0.0,"$15,181.0",$0.0


In [49]:
Final_Pivot.to_excel(Path+'Output\\Summary\\Shipment Summary.xlsx',index=False)
pivot_qty.to_excel(Path+'Output\\Mail\\Shipment by Qty.xlsx',index=False)
pivot_cost.to_excel(Path+'Output\\Mail\\Shipment by Cost.xlsx',index=False)

In [50]:
print('**************************************************************************************\n')
print('                 Shipment Completed................!)\n')
print('**************************************************************************************\n')

**************************************************************************************

                 Shipment Completed................!)

**************************************************************************************



## ***************************************************************************************************************

## ******************************** Forecast (regional_tls) File ********************************

In [51]:
print('**************************************************************************************\n')
print('                 Forecast Process (New Demand Started................!)\n')
print('**************************************************************************************\n')

**************************************************************************************

                 Forecast Process (New Demand Started................!)

**************************************************************************************



In [52]:
####### Read Global DOC
TLS = Path + 'Input\\'
file_list = glob.glob(doc_path + '????????_???_regional_tls.csv')
if file_list:
    TLS = pd.read_csv(file_list[0])  
else:
    print("No TLS file found.")

C:\Users\nn385006\AppData\Local\Temp\2\ipykernel_18372\3639996027.py:5: DtypeWarning: Columns (0,3,4,5,8,9,10,11,12,16,17,24,27,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  TLS = pd.read_csv(file_list[0])


In [53]:
TLS = TLS[
    (TLS['Country Code'] == 'US') &
    (TLS['Prod Grp'] == 'ATM') &
    (TLS['Offer PF'].str.contains('MF|CD', na=False)) &
    (TLS['Data Series'] == 'NEW DEMAND') &
    (TLS['Item Type'].str.contains('Unassigned|Unit', na=False))
]

### Update Cost 

In [54]:
val = ['Qty', 'ASP']

for x in val:
    TLS[x] = pd.to_numeric(TLS[x], errors='coerce').astype('Int64')

TLS['Class'] = pd.to_numeric(TLS['Class'], errors='coerce').astype('Int64')
TLS['Class'] = TLS['Class'].astype(str)

TLS['Cost'] = TLS['Qty'] * TLS['ASP']
TLS['Cost'] = TLS['Cost'].round(0)

In [58]:
#TLS['Ship Month'] = pd.to_datetime(TLS['Date'], format='%d-%m-%Y').dt.strftime('%Y%m')
TLS['Ship Month'] = pd.to_datetime(TLS['Date'])
#TLS['Ship Month'] = pd.to_datetime(TLS['Date']).dt.strftime('%Y%m')

In [59]:
#TLS.to_csv(Path+'Output\\output.csv',index=False)

## Grouping 

In [60]:
Pivot = TLS.copy()

pivot_qty = Pivot.pivot_table(
    index=['Plant','Offer PF','Class','Key Account'],
    columns='Ship Month',
    values='Qty',
    aggfunc='sum',
    fill_value=0
)



cost_sum = TLS.groupby(['Plant','Offer PF','Class','Key Account'])['Cost'].sum()
cost_asp_sum = TLS.groupby(['Plant','Offer PF','Class','Key Account'])['ASP'].sum()

Final_Pivot = pivot_qty.merge(cost_sum, on=['Plant','Offer PF','Class','Key Account']).reset_index()
Final_Pivot = Final_Pivot.merge(cost_asp_sum, on=['Plant','Offer PF','Class','Key Account']).reset_index()

In [61]:
Final_Pivot.insert(5,'Cost',Final_Pivot.pop('Cost'))
Final_Pivot.insert(6,'ASP',Final_Pivot.pop('ASP'))

val = {'Key Account':'Key Acc','ASP':'$ ASP'}

Final_Pivot.rename(columns=val, inplace=True)

Final_Pivot['Data series'] = 'New Demand'

Final_Pivot.insert(0,'Data series',Final_Pivot.pop('Data series'))

### Qty Pivot 

In [62]:
Pivot = TLS.copy()

Pivot['Data Series'] = 'New Demand'

pivot_qty = Pivot.pivot_table(
    index=['Data Series','Plant'],
    columns='Ship Month',
    values='Qty',
    aggfunc='sum',
    fill_value=0
).reset_index()


In [63]:
pivot_qty

Ship Month,Data Series,Plant,2025-01-11 00:00:00,2025-01-12 00:00:00,2026-01-01 00:00:00,2026-01-02 00:00:00,2026-01-03 00:00:00,2026-01-04 00:00:00,2026-01-05 00:00:00,2026-01-06 00:00:00,2026-01-07 00:00:00,2026-01-08 00:00:00,2026-01-09 00:00:00,2026-01-10 00:00:00,2026-01-11 00:00:00,2026-01-12 00:00:00,2027-01-01 00:00:00,2027-01-02 00:00:00,2027-01-03 00:00:00,2027-01-04 00:00:00
0,New Demand,CHE,2,581,73,286,584,770,959,977,676,772,857,737,770,789,870,943,782,988
1,New Demand,EC,4,1,9,41,69,90,91,58,76,58,66,55,113,122,106,113,112,158


#### $ Value Cost Pivot 

In [64]:
Pivot = TLS.copy()

Pivot['Data Series'] = 'New Demand'

pivot_cost = Pivot.pivot_table(
    index=['Data Series','Plant'],
    columns='Ship Month',
    values='Cost',
    aggfunc='sum',
    fill_value=0
).reset_index()


for col in pivot_cost.columns:
    if col not in ['Data Series', 'Plant']:
        pivot_cost[col] = pivot_cost[col].apply(lambda x: f"${x:,}")



In [65]:
pivot_cost

Ship Month,Data Series,Plant,2025-01-11 00:00:00,2025-01-12 00:00:00,2026-01-01 00:00:00,2026-01-02 00:00:00,2026-01-03 00:00:00,2026-01-04 00:00:00,2026-01-05 00:00:00,2026-01-06 00:00:00,2026-01-07 00:00:00,2026-01-08 00:00:00,2026-01-09 00:00:00,2026-01-10 00:00:00,2026-01-11 00:00:00,2026-01-12 00:00:00,2027-01-01 00:00:00,2027-01-02 00:00:00,2027-01-03 00:00:00,2027-01-04 00:00:00
0,New Demand,CHE,$0,"$17,754,897","$1,084,569","$6,351,957","$11,271,867","$15,092,425","$18,545,645","$20,918,103","$14,544,736","$14,655,905","$17,714,028","$12,842,281","$14,384,800","$16,363,202","$14,245,524","$17,766,931","$13,024,447","$16,470,453"
1,New Demand,EC,$0,"$28,860","$44,056","$752,145","$1,079,995","$1,064,064","$1,005,271","$780,541","$1,069,593","$637,508","$869,054","$867,126","$1,271,220","$1,530,190","$1,298,612","$1,358,004","$1,266,830","$1,679,068"


In [66]:
Final_Pivot.to_excel(Path+'Output\\Summary\\New Demand by Summary.xlsx',index=False)
pivot_qty.to_excel(Path+'Output\\Mail\\New Demand by Qty.xlsx',index=False)
pivot_cost.to_excel(Path+'Output\\Mail\\New Demand by Cost.xlsx',index=False)

In [67]:
print('**************************************************************************************\n')
print('                 Forecast Process (New Demand Completed................!)\n')
print('**************************************************************************************\n')

**************************************************************************************

                 Forecast Process (New Demand Completed................!)

**************************************************************************************



In [68]:
print('**************************************************************************************\n')
print('................Automation Completed................\n')
print('**************************************************************************************\n')

**************************************************************************************

................Automation Completed................

**************************************************************************************

